In [1]:
import xarray as xr
import numpy as np

f1 = "/Volumes/Jedha/cloud_locking/B1850_c201_CTL/B1850_c201_CTL.cam.h0.0001-01.ncrcat.TMQ.nc"
f2 = "/Volumes/Jedha/cloud_locking/B1850_c201_CLOCK/B1850_c201_CLOCK.cam.h0.0001-01.ncrcat.TMQ.nc"

ds1 = xr.open_dataset(f1, decode_times=False)
ds2 = xr.open_dataset(f2, decode_times=False)


In [2]:
def cesm_correct_time(ds):
    """Given a Dataset, check for time_bnds,
       and use avg(time_bnds) to replace the time coordinate.
       Purpose is to center the timestamp on the averaging inverval.   
       NOTE: ds should have been loaded using `decode_times=False`
    """
    assert 'time_bnds' in ds
    assert 'time' in ds
    correct_time_values = ds['time_bnds'].mean(dim='nbnd')
    # copy any metadata:
    correct_time_values.attrs = ds['time'].attrs
    ds = ds.assign_coords({"time": correct_time_values})
    ds = xr.decode_cf(ds)  # decode to datetime objects
    return ds

In [3]:
ds1 = cesm_correct_time(ds1)
ds2 = cesm_correct_time(ds2)

prw1 = ds1['TMQ']
prw2 = ds2['TMQ']

In [4]:
prw1.shape

(361, 192, 288)

In [6]:
# Alternative is to use two-sided t-statistic
from scipy import stats

# 1st start from the full data, and let's just see if the means are different

v1 = prw1.values.flatten()
# vs1 = v1[~np.isnan(v1)]

v2 = prw2.values.flatten()
# vs2 = v2[~np.isnan(v2)]
print("get test stats")
tstat, pval = stats.ttest_ind(v1,v2, equal_var=False)
print(f"The t-statistic is {tstat}, the p-value is {pval}; If pvalue is small, the means are different. Check {pval < 0.05}")

get test stats
The t-statistic is 4.850194709683481, the p-value is 1.23340845646681e-06; If pvalue is small, the means are different. Check True


In [7]:
# LAND
land_file = xr.open_dataset("/Users/brianpm/Dropbox/Data/cesm2_f09_land.nc")
LAND = land_file['LANDFRAC'].squeeze()



In [8]:
# 2nd move to tropical ocean, removing the land values from the data

prw1to = prw1.where(LAND <= 0).sel(lat=slice(-30,30))
prw2to = prw2.where(LAND <= 0).sel(lat=slice(-30,30))

v1 = prw1to.values.flatten()
vs1 = v1[~np.isnan(v1)]

v2 = prw2to.values.flatten()
vs2 = v2[~np.isnan(v2)]
print("get test stats")
tstat, pval = stats.ttest_ind(vs1,vs2, equal_var=False)
print(f"The t-statistic is {tstat}, the p-value is {pval}; If pvalue is small, the means are different. Check {pval < 0.05}")

get test stats
The t-statistic is -0.598293224327227, the p-value is 0.5496443126958364; If pvalue is small, the means are different. Check False


In [12]:
# JUST DOUBLE CHECK USING ANNUAL MEANS:
prw1_ann = prw1.sel(time=slice(None, '0030-12-31')).groupby('time.year').mean(dim='time')
prw1_ann

<xarray.DataArray 'TMQ' (year: 30, lat: 192, lon: 288)>
array([[[0.47864708, 0.47864708, 0.47864708, ..., 0.4786471 ,
         0.47864708, 0.47864708],
        [0.49533546, 0.49419037, 0.49303916, ..., 0.49881336,
         0.4976623 , 0.49647847],
        [0.51106685, 0.50870514, 0.5063328 , ..., 0.51827115,
         0.5158536 , 0.5134405 ],
        ...,
        [5.968722  , 5.9718227 , 5.9748588 , ..., 5.9601626 ,
         5.962973  , 5.965872  ],
        [5.886055  , 5.886575  , 5.887053  , ..., 5.884578  ,
         5.8850856 , 5.885533  ],
        [5.845066  , 5.8450646 , 5.845064  , ..., 5.84506   ,
         5.845059  , 5.8450627 ]],

       [[0.4789008 , 0.4789008 , 0.4789008 , ..., 0.4789008 ,
         0.4789008 , 0.4789008 ],
        [0.5030179 , 0.5020256 , 0.500975  , ..., 0.5060354 ,
         0.505032  , 0.50400615],
        [0.5255371 , 0.523371  , 0.5211572 , ..., 0.53200924,
         0.52983564, 0.5276721 ],
...
        [5.963993  , 5.9646015 , 5.965137  , ..., 5.960861  ,
         5.962045  , 5.963209  ],
        [5.857887  , 5.8585906 , 5.859268  , ..., 5.855503  ,
         5.856369  , 5.857245  ],
        [5.822319  , 5.822283  , 5.822286  , ..., 5.8222218 ,
         5.8223224 , 5.822235  ]],

       [[0.45148674, 0.45148706, 0.45148668, ..., 0.45148683,
         0.45148692, 0.45148697],
        [0.46331945, 0.46235874, 0.4613373 , ..., 0.46628645,
         0.4652872 , 0.46427965],
        [0.4723911 , 0.47022843, 0.46805644, ..., 0.47899893,
         0.47677472, 0.4745725 ],
        ...,
        [5.8539453 , 5.860789  , 5.867842  , ..., 5.8337007 ,
         5.8404408 , 5.847132  ],
        [5.8338094 , 5.8369    , 5.8401217 , ..., 5.8239493 ,
         5.8271995 , 5.830459  ],
        [5.8193893 , 5.8193073 , 5.8194156 , ..., 5.819385  ,
         5.8194237 , 5.8194146 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1 2 3 4 5 6 7 8 9 10 ... 21 22 23 24 25 26 27 28 29 30

In [13]:
prw2_ann = prw2.sel(time=slice(None, '0030-12-31')).groupby('time.year').mean(dim='time')
prw2_ann

<xarray.DataArray 'TMQ' (year: 26, lat: 192, lon: 288)>
array([[[0.42876038, 0.42876038, 0.42876038, ..., 0.42876038,
         0.42876038, 0.42876038],
        [0.44442272, 0.443441  , 0.44241557, ..., 0.44734025,
         0.44637886, 0.44539082],
        [0.4610751 , 0.4589344 , 0.45675847, ..., 0.46736348,
         0.4652853 , 0.4631864 ],
        ...,
        [5.77199   , 5.774713  , 5.777471  , ..., 5.7637005 ,
         5.766484  , 5.769245  ],
        [5.7638836 , 5.7653985 , 5.7669754 , ..., 5.7594066 ,
         5.7610135 , 5.7624874 ],
        [5.790001  , 5.790001  , 5.789989  , ..., 5.7900014 ,
         5.7900004 , 5.7900014 ]],

       [[0.40662578, 0.40662584, 0.40662584, ..., 0.40662578,
         0.40662578, 0.40662578],
        [0.42029855, 0.41930684, 0.41827273, ..., 0.42327484,
         0.42227277, 0.4212748 ],
        [0.435509  , 0.4333022 , 0.4310651 , ..., 0.44201374,
         0.4398539 , 0.43768954],
...
        [5.8225136 , 5.8277135 , 5.832868  , ..., 5.8067894 ,
         5.8120136 , 5.8172555 ],
        [5.7980695 , 5.8006725 , 5.8032107 , ..., 5.7900643 ,
         5.792726  , 5.7954288 ],
        [5.7707253 , 5.7707257 , 5.770703  , ..., 5.7707024 ,
         5.7707257 , 5.7707257 ]],

       [[0.44040117, 0.44040117, 0.44040117, ..., 0.44040117,
         0.44040117, 0.44040117],
        [0.45983946, 0.45883396, 0.4577832 , ..., 0.46286848,
         0.46185783, 0.46083036],
        [0.482663  , 0.4803497 , 0.47801754, ..., 0.48960814,
         0.4872931 , 0.48496917],
        ...,
        [5.502789  , 5.5055404 , 5.5081325 , ..., 5.4942856 ,
         5.4972205 , 5.4999886 ],
        [5.388485  , 5.390102  , 5.3917317 , ..., 5.383963  ,
         5.385444  , 5.3869596 ],
        [5.299991  , 5.2999907 , 5.29999   , ..., 5.2999907 ,
         5.299991  , 5.299991  ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1 2 3 4 5 6 7 8 9 10 ... 17 18 19 20 21 22 23 24 25 26

In [14]:
# 3rd annual means for tropical ocean, removing the land values from the data

prw1to = prw1_ann.where(LAND <= 0).sel(lat=slice(-30,30))
prw2to = prw2_ann.where(LAND <= 0).sel(lat=slice(-30,30))

v1 = prw1to.values.flatten()
vs1 = v1[~np.isnan(v1)]

v2 = prw2to.values.flatten()
vs2 = v2[~np.isnan(v2)]
print("get test stats")
tstat, pval = stats.ttest_ind(vs1,vs2, equal_var=False)
print(f"The t-statistic is {tstat}, the p-value is {pval}; If pvalue is small, the means are different. Check {pval < 0.05}")

get test stats
The t-statistic is -0.17301690673118816, the p-value is 0.8626381875698816; If pvalue is small, the means are different. Check False
